## Preprocessing

In [60]:
#!pip install keras

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pickle

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [31]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_drop = application_df.drop(["EIN", "NAME"], axis=1)
app_drop.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [32]:
# Determine the number of unique values in each column.
app_drop.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [33]:
# Look at APPLICATION_TYPE value counts for binning
app_drop["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# code derived from outside source aka Daniel-RG-Bill
app_types_replace = app_drop.apply(lambda x: x.mask(x.map(x.value_counts())<1065, 'Other') if x.name=='APPLICATION_TYPE' else x)
app_types_replace["APPLICATION_TYPE"].value_counts()


# # Replace in dataframe
# for app in application_types_to_replace:
#     app_drop['APPLICATION_TYPE'] = app_drop['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
# app_drop['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [35]:
app_types_replace["APPLICATION_TYPE"].nunique()

6

In [36]:
# Look at CLASSIFICATION value counts for binning
app_drop["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [37]:
list_classification=app_drop['CLASSIFICATION'].value_counts()
list_class=list_classification.tolist()
list_class

[17326,
 6074,
 4837,
 1918,
 1883,
 777,
 287,
 194,
 116,
 114,
 104,
 95,
 75,
 58,
 50,
 36,
 34,
 32,
 32,
 30,
 20,
 18,
 16,
 15,
 15,
 14,
 11,
 10,
 10,
 9,
 9,
 7,
 6,
 6,
 6,
 5,
 5,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [38]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# app_types_replace[app_types_replace['CLASSIFICATION'].value_counts() > 1]

In [39]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
app_types_replace = app_types_replace.apply(lambda x: x.mask(x.map(x.value_counts())<1883, 'Other') if x.name=='CLASSIFICATION' else x)
app_types_replace["CLASSIFICATION"].value_counts()

# # Replace in dataframe
# for cls in classifications_to_replace:
#     application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [40]:
app_types_replace["CLASSIFICATION"].nunique()

6

In [41]:
# Check D-types
app_types_replace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [42]:
app_types_replace.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0            Other       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [43]:
# Convert categorical data to numeric with `pd.get_dummies`
app_numeric = pd.get_dummies(app_types_replace)
app_numeric

STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       1   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                         0                    0                    0   
1                         0                    1                    0   
2                         0                    0                    0   
3                         0                    1                    0   
4                         0                    1                    0   
...                     ...                  ...                  ...   
34294                     0                    0                    1   
34295                     0                    0                    1   
34296                     0                    1                    0   
34297                     0                    0                    0   
34298                     0                    1                    0   

       APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  AFFILIATION_CompanySponsored  \
0                        0                    0                             0   
1                        0                    0                             0   
2                        1                    0                             1   
3                        0                    0                             1   
4                        0                    0                             0   
...                    ...                  ...                           ...   
34294                    0                    0                             0   
34295                    0                    0                             1   
34296                    0                    0                             1   
34297                    1                    0                             0   
34298                    0                    0                             0   

       ...  INCOME_AMT_1-9999  INCOME_AMT_10000-24999  \
0      ...                  0                       0   
1      ...                  1                       0   
2      ...                  0                       0   
3      ...                  0                       1   
4      ...                  0                       0   
...    ...                ...                     ...   
34294  ...                  0                       0   
34295  ...                  0                       0   
34296  ...                  0                       0   
34297  ...                  0                       0   
34298  ...                  0                       0   

       INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  \
0                             0                   0                 0   
1                             0                   0                 0   
2                             0                   0                 0   
3                             0                   0                 0   
4                             1                   0                 0   
...                         ...                 ...               ...   
34294                         0                   0                 0   
34295                         0                   0                 0   
34296                 

In [44]:
# confirm columns 
app_numeric.columns


Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
 

In [45]:
# Split our preprocessed data into our features and target arrays
y = app_numeric["IS_SUCCESSFUL"]
X = app_numeric.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [46]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=40))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 15)                615       
                                                                 
 dense_20 (Dense)            (None, 25)                400       
                                                                 
 dense_21 (Dense)            (None, 1)                 26        
                                                                 
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [66]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6022 - accuracy: 0.6951
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5642 - accuracy: 0.7255
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5596 - accuracy: 0.7266
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5573 - accuracy: 0.7296
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7288
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7289
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7297
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7291
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7315
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7300

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7312 - 250ms/epoch - 933us/step
Loss: 0.5576657652854919, Accuracy: 0.731195330619812


In [69]:
# Export our model to HDF5 file
filename='AlphabetSoupCharity.h5'
pickle.dump(fit_model,open(filename,'wb'))

INFO:tensorflow:Assets written to: ram://e647da2a-cf04-49f3-a716-c1ef8efabb05/assets
